In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.lap.track import *
from toolkit.mmd import MMD
from toolkit.steady_state_solver import LS_Solver, Min_Solver, Iterative_Solver, Parachute
from toolkit.steady_state_solver.min_optimize import backup_loss_func

In [ ]:
v_avg = 15
max_beta = 30
max_delta = 30
size = 30
use_lin = True

In [ ]:
# car = Car()
rsd = 0.25
total_roll_stiffness = 900 * FTLB_TO_NM
car = Car(front_axle_weight=0.4725)
car.z_f = 0.13 * IN_TO_M
car.z_r = 1.0 * IN_TO_M
car.cdA = 1.33
car.clA = 3.43
car.cg_height=10.7*IN_TO_M
car.front_axle_downforce = 0.48
car.k_f = total_roll_stiffness * rsd
car.k_r = total_roll_stiffness * (1-rsd)
car.k_c = 1000000
# car.diff_model = "locked"
car.update_car()
car.set_lltd()
# car.fast_mf = None
# solver = LS_Solver()
solver = Min_Solver()
# solver = Iterative_Solver()
# solver = Parachute()

In [ ]:
long_g = 0.0
delta_x = 1.0
beta_x = 1.0
mu_corr = 0.65

In [ ]:
yaw_it, cn_it = 0.0, 0.0
drag = 0
max_f, max_r = car.max_front_brake_torque / -car.mf_tire.UNLOADED_RADIUS, car.max_rear_brake_torque / -car.mf_tire.UNLOADED_RADIUS

max_tractive_force = car.find_tractive_force(vel=v_avg, use_aero=False)
drag = 0.5 * 1.225 * v_avg**2 * car.cd * car.A

args = (car, long_g, v_avg, delta_x, beta_x, mu_corr, drag, max_f, max_r, max_tractive_force)

In [ ]:
lat_acc_range = np.linspace(-30, 30, 100)
fx_range = np.linspace(2*(max_f + max_r) - drag, max_tractive_force, 100)

error = np.zeros([len(lat_acc_range), len(fx_range)])
ay_error = np.zeros([len(lat_acc_range), len(fx_range)])
long_error = np.zeros([len(lat_acc_range), len(fx_range)])
for i, lat_acc in enumerate(lat_acc_range):
    for j, fx in enumerate(fx_range):
        error[i, j] = backup_loss_func([lat_acc, fx], car, v_avg, long_g, np.deg2rad(delta_x), np.deg2rad(beta_x), mu_corr, drag, max_f, max_r, max_tractive_force) # ay_error[i, j], long_error[i, j]

In [ ]:
# plot the error heatmap
fig = go.Figure()
fig.add_trace(go.Heatmap(z=error, x=fx_range, y=lat_acc_range, colorscale='Viridis'))
fig.update_layout(title='Error Heatmap', width=800, height=800)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=ay_error, x=fx_range, y=lat_acc_range, colorscale='Viridis'))
fig.update_layout(title='Error Heatmap', width=800, height=800)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=long_error, x=fx_range, y=lat_acc_range, colorscale='Viridis'))
fig.update_layout(title='Error Heatmap', width=800, height=800)
fig.show()